In [1]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [2]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df.index = df.index + 1
df = df.dropna()

In [3]:
df_normal = pd.read_csv('data.txt', sep=" ", header=None)
df_normal.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]

In [4]:
df_normal = df_normal.loc[df_normal['class']  == 'NORM']
df_normal = df_normal.reset_index(drop=True)

In [26]:
df_normal

,ID,background_tissue,class,severity,x,y,rad
0,mdb003,D,NORM,NaN,NaN,NaN,NaN
1,mdb004,D,NORM,NaN,NaN,NaN,NaN
2,mdb006,F,NORM,NaN,NaN,NaN,NaN
3,mdb007,G,NORM,NaN,NaN,NaN,NaN
4,mdb008,G,NORM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
202,mdb318,D,NORM,NaN,NaN,NaN,NaN
203,mdb319,D,NORM,NaN,NaN,NaN,NaN
204,mdb320,D,NORM,NaN,NaN,NaN,NaN
205,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [ ]:
df['severity'].value_counts()

In [ ]:
im = Image.open('all-mias/mdb001.pgm') # PILLOW image processing

In [ ]:
im.getpixel( (535,425) )

In [ ]:
last = 'start'

In [ ]:
def img_draw(image, x, y, r, c):
    if c=='M':
        color = 'red'
    else:
        color = 'blue'
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, outline=color, width=5)

In [ ]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [37]:
mean_B = []
var_B = []
mean_M = []
var_M = []
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    if (c == 'M'):
        mean_M.append(cv2.meanStdDev(im, mask = mask)[0][0])
        var_M.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    else:
        mean_B.append(cv2.meanStdDev(im, mask = mask)[0][0])
        var_B.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))

In [38]:
df_normal

,ID,background_tissue,class,severity,x,y,rad
2,mdb003,D,NORM,NaN,NaN,NaN,NaN
3,mdb004,D,NORM,NaN,NaN,NaN,NaN
6,mdb006,F,NORM,NaN,NaN,NaN,NaN
7,mdb007,G,NORM,NaN,NaN,NaN,NaN
8,mdb008,G,NORM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
325,mdb318,D,NORM,NaN,NaN,NaN,NaN
326,mdb319,D,NORM,NaN,NaN,NaN,NaN
327,mdb320,D,NORM,NaN,NaN,NaN,NaN
328,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [42]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),50,(255,0,0),3)
        cv2.imshow('image', img)
        mouseX,mouseY = x,y

df_normal = pd.read_csv('data.txt', sep=" ", header=None)
df_normal.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df_normal = df_normal.loc[df_normal['class']  == 'NORM']
df_normal = df_normal.reset_index(drop=True)

i = 0

path = "marked_norm"
while(1):
    img = cv2.imread('all-mias/{name}.pgm'.format(name=df_normal.iloc[i]['ID']))
    cv2.imshow('image',img)
    #rad = [50]
    cv2.setMouseCallback('image', draw_circle)
    k = cv2.waitKey(0) & 0xFF
    if k == 27:
        break
    elif k == ord('a'):
        print(mouseX,mouseY)
    elif k == ord('s'):
        cv2.imwrite(os.path.join(path , '{name}.jpg'.format(name=df_normal.iloc[i]['ID'])), img)
    i += 1
cv2.destroyAllWindows()

In [14]:
img = cv2.imread('all-mias/mdb001.pgm')
cv2.imshow("image", img)
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
im = cv2.imread('all-mias/mdb001.pgm')

In [ ]:
h,w,d = im.shape

In [ ]:
mask = np.zeros((h,w), np.uint8)

In [ ]:
cv2.circle(mask,(535,1024-425),197,1,thickness=-1)

In [ ]:
masked_data = cv2.bitwise_and(im, im, mask=mask)

In [ ]:
cv2.imshow("masked", masked_data)
cv2.waitKey(0)

In [ ]:
cv2.mean(im, mask = mask)

In [ ]:
cv2.meanStdDev(im, mask = mask)[0][0]

In [ ]:
cv2.imwrite("masked.png", masked_data)